In [8]:
from bs4 import BeautifulSoup
import requests
import json

# Parsing
## Get list of all the sections

In [9]:
headers = requests.utils.default_headers()
url = "https://www.tesco.com/groceries/en-GB/shop/?include-children=true"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
result = soup.find_all("script", type="application/ld+json")
list_ = json.loads(result[0].string)
sections_urls_list = []
for next_ in list_:
    if next_["@type"] == "ItemList":
        elements_list = next_["itemListElement"]
        sections_urls_list = list(map(lambda x: x["url"], elements_list))
        break
print(sections_urls_list)



['https://www.tesco.com/groceries/en-GB/shop/fresh-food', 'https://www.tesco.com/groceries/en-GB/shop/bakery', 'https://www.tesco.com/groceries/en-GB/shop/frozen-food', 'https://www.tesco.com/groceries/en-GB/shop/food-cupboard', 'https://www.tesco.com/groceries/en-GB/shop/drinks', 'https://www.tesco.com/groceries/en-GB/shop/baby', 'https://www.tesco.com/groceries/en-GB/shop/health-and-beauty', 'https://www.tesco.com/groceries/en-GB/shop/pets', 'https://www.tesco.com/groceries/en-GB/shop/household', 'https://www.tesco.com/groceries/en-GB/shop/home-and-ents', 'https://www.tesco.com/groceries/en-GB/shop/easter', 'https://www.tesco.com/groceries/en-GB/shop/inspiration-and-events']


## Handling content inside sections

In [57]:
#parse for products urls on the page
def parse_page_content(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    result = soup.find_all("script", type="application/ld+json")
    list_ = json.loads(result[0].string)
    for next_ in list_:
        if next_["@type"] == "ItemList":
            elements_list = next_["itemListElement"]
            sections_urls_list = list(map(lambda x: x["url"], elements_list))
            break
    return sections_urls_list

def find_last_page_index(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    #try to get the last page index:
    result = soup.find_all("li", class_="pagination-btn-holder")
    #-1 is going back to results, -2 - is the last page
    last_page = result[-2].find_all("span")[0].string
    return int(last_page)

    
#Finds last page and call parsing for each page
def parse_section_content(url):
    last_page = find_last_page_index(url)
    #cause I don't want to scrape everything, most probably we will need to have some delays for prevent bot detection checks
    i = 0
    result_set = set()
    url = url+"?page=" 
    for next_page in range(1, last_page+1):
        page_url = url + str(next_page)
        local_result = parse_page_content(page_url)
        result_set = result_set.union(local_result)
        i = i + 1
        if i == 3:
            return result_set
    return result_set
        


In [58]:
for next_url in sections_urls_list:
    data = parse_section_content(next_url+"/all")
    break


In [59]:
print(len(data))

2946


# Sainsbury's parsing

In [2]:
main_sainsbury_url = 'https://www.sainsburys.co.uk/shop/gb/groceries'
sainsbury_categories_urls = [
'https://www.sainsburys.co.uk/shop/gb/groceries/dietary-and-lifestyle/seeall?fromMegaNav=1#langId=44&storeId=10151&catalogId=10241&categoryId=453878&parent_category_rn=&top_category=453878&pageSize=60&orderBy=FAVOURITES_FIRST&searchTerm=&catSeeAll=true&beginIndex=0&categoryFacetId1=453878&categoryFacetId2=',
    'https://www.sainsburys.co.uk/shop/gb/groceries/fruit-veg/seeall?fromMegaNav=1#langId=44&storeId=10151&catalogId=10241&categoryId=12518&parent_category_rn=&top_category=12518&pageSize=60&orderBy=FAVOURITES_FIRST&searchTerm=&catSeeAll=true&beginIndex=0&categoryFacetId1=12518&categoryFacetId2=',
    'https://www.sainsburys.co.uk/shop/gb/groceries/meat-fish/seeall?fromMegaNav=1#langId=44&storeId=10151&catalogId=10241&categoryId=13343&parent_category_rn=&top_category=13343&pageSize=60&orderBy=FAVOURITES_FIRST&searchTerm=&catSeeAll=true&beginIndex=0&categoryFacetId1=13343&categoryFacetId2=',
    'https://www.sainsburys.co.uk/shop/gb/groceries/dairy-eggs-and-chilled/seeall?fromMegaNav=1#langId=44&storeId=10151&catalogId=10241&categoryId=428866&parent_category_rn=&top_category=428866&pageSize=60&orderBy=FAVOURITES_FIRST&searchTerm=&catSeeAll=true&beginIndex=0&categoryFacetId1=428866&categoryFacetId2=',
    'https://www.sainsburys.co.uk/shop/gb/groceries/bakery/seeall?fromMegaNav=1#langId=44&storeId=10151&catalogId=10241&categoryId=12320&parent_category_rn=&top_category=12320&pageSize=60&orderBy=FAVOURITES_FIRST&searchTerm=&catSeeAll=true&beginIndex=0&categoryFacetId1=12320&categoryFacetId2=',
    'https://www.sainsburys.co.uk/shop/gb/groceries/frozen-/seeall?fromMegaNav=1#langId=44&storeId=10151&catalogId=10241&categoryId=218831&parent_category_rn=&top_category=218831&pageSize=60&orderBy=FAVOURITES_FIRST&searchTerm=&catSeeAll=true&beginIndex=0&categoryFacetId1=218831&categoryFacetId2=',
    'https://www.sainsburys.co.uk/shop/gb/groceries/food-cupboard/seeall?fromMegaNav=1#langId=44&storeId=10151&catalogId=10241&categoryId=12422&parent_category_rn=&top_category=12422&pageSize=60&orderBy=FAVOURITES_FIRST&searchTerm=&catSeeAll=true&beginIndex=0&categoryFacetId1=12422&categoryFacetId2=',
'https://www.sainsburys.co.uk/shop/CategorySeeAllView?listId=&catalogId=10241&searchTerm=&beginIndex=0&pageSize=120&orderBy=FAVOURITES_FIRST&top_category=&langId=44&storeId=10151&categoryId=12192&promotionId=&parent_category_rn='
]

In [3]:
import re

def link_to_next_page_from_url(url):
    req = requests.get(url, [])
    soup = BeautifulSoup(req.content, 'html.parser')
    return link_to_next_page(soup)


def link_to_next_page(soup):
    result = soup.find_all("li", class_="next")
    data = result[0].find_all('a')
    if len(data) > 0:
        return data[0].get('href')
    return ''

def links_to_products_from_url(url):
    req = requests.get(url, [])
    soup = BeautifulSoup(req.content, 'html.parser')
    return links_to_products(soup)

def links_to_products(soup):
    result = soup.find_all("div", class_="productNameAndPromotions")
    result_set = set()
    for next_ in result:
        data = next_.find_all('a', attrs={'href': re.compile("^http")})
        if len(data) > 0:
            result_set.add(data[0].get('href'))
    return result_set

In [4]:
def parse_category(url):
    result_set = set()
    next_url = url

    while len(next_url) > 0:
        req = requests.get(next_url, [])
        soup = BeautifulSoup(req.content, 'html.parser')
        next_url = link_to_next_page(soup)
        result_set = result_set.union(links_to_products(soup))
        parse_product_from_url(result_set.pop())
    return result_set


In [35]:
from selenium import webdriver

def parse_product_from_url(url):
    driver = webdriver.Firefox(executable_path = '/usr/local/bin/geckodriver')
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html)
    driver.close()
    driver.quit()

    result = soup.find_all("div", class_="ln-c-card pd-details ln-c-card--soft")
    print(result)
#     nutrition = result[0].find_all('div', class_='nutritionalContentSummary')
#     print(nutrition[0])
#     country = result[0].find_all('div', class_='productText')
#     print(country[0])

    

In [36]:
parse_product_from_url('https://www.sainsburys.co.uk/gol-ui/product/all-butter-spreads/sainsburys-organic-unsalted-british-butter-250g')

[<div class="ln-c-card pd-details ln-c-card--soft"><div><productcontent xmlns:a="http://www.inspire-js.com/SOL">
<htmlcontent contentpath="/Content/media/html/products/label//_label_inspire.html" outputmethod="xhtml">
<h3 class="productDataItemHeader">Description</h3>
<div class="productText">
<p>Organic unsalted butter</p>
<p>
             
            </p>
<p>
</p><p>Cream made from organic cows' milk...   ...churned while it's fresh.</p>
<p></p>
</div>
<h3 class="productDataItemHeader">Nutrition</h3>
<div class="productText">
<div class="nutritionalContentSummary">
<h2 class="reference">Per 2 teaspoons (10g) Typical Values</h2>
<ul class="lozengeBlock">
<li class="lozenge energy energy">
<div class="lozengeHeaderSection">
<h3 class="lozengeTitle">ENERGY</h3>
<p>309kJ</p>
<p>75kcal</p>
</div>
<div class="percentage">
<p>4%</p>
</div>
<p class="access">white</p>
</li>
<li class="lozenge high fat">
<div class="lozengeHeaderSection">
<h3 class="lozengeTitle">FAT</h3>
<p>8.3g</p>
</div>


In [19]:
result_set = set()
for next_url in sainsbury_categories_urls:
    result_set = result_set.union(parse_category(next_url))

<html lang="en"><head><meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><title>Sainsbury's British Butter Unsalted, SO Organic 250g | Sainsbury's</title><meta content="width=device-width,initial-scale=1,shrink-to-fit=no" name="viewport"/><meta content="#000000" name="theme-color"/><link href="/gol-ui/manifest.json" rel="manifest"/><link href="/favicon.ico" rel="shortcut icon"/><link href="/gol-ui/static/css/app.5ded35e9.css" rel="stylesheet"/><link href="https://f.monetate.net" rel="dns-prefetch"/><link href="https://sb.monetate.net" rel="dns-prefetch"/><meta data-jc="82" data-jc-version="r20200401"/><script src="https://js-agent.newrelic.com/nr-spa-1071.min.js"></script><script async="" src="https://rules.quantcount.com/rules-p-84W2U6IHUzMYI.js"></script><script async="" src="https://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript"></script><script async="" src="https://www.google.com/pagead/conversion_async.js" type="text/javascrip

TypeError: 'NoneType' object is not iterable